In [ ]:
# Kevin Richard
# Artist Multi-Classifiaction

In [ ]:
import sys
!{sys.executable} -m pip install opencv-python
import cv2
import numpy as np

import os
from IPython.display import display, Image

import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [ ]:

# Change to appropriate file path
Train_Folder ='/Painting/Train/'
Validation_Folder ='/Painting/Validation/'


Artists = os.listdir(Train_Folder)

image_list = []
for Artist in Artists:
    path = os.path.join(Train_Folder, Artist)
    image_files = [f for f in os.listdir(path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif'))]
    for image_file in image_files:
        image_path = os.path.join(path, image_file)
        image_list.append([Image(filename=image_path), Artist])
        display(Image(filename=image_path))



In [ ]:
display(image_list[1][1])

In [ ]:
# Training Data
Train_list2 = []
for Artist in Artists:
    path = os.path.join(Train_Folder, Artist)
    for image_file in os.listdir(path):
        image_path = os.path.join(path, image_file)
        img = cv2.imread(image_path)
        resized_image = cv2.resize(img, (256, 256))
        Train_list2.append([resized_image, Artist])


In [ ]:
# Validation Data
Validation_list = []
for image_file in os.listdir(Validation_Folder):
    image_path = os.path.join(Validation_Folder, image_file)
    img = cv2.imread(image_path)
    artist = image_file.rsplit('_', 1)[0]
    resized_image = cv2.resize(img, (256, 256))
    Validation_list.append([resized_image, artist])

In [ ]:
X =[]
y=[]
# random.shuffle(Train_list2)
for row in Train_list2:
    X.append(row[0]/255.0)
    y.append(row[1])

X_t = []
y_t = []

for row in Validation_list:
    X_t.append(row[0]/255.0)
    y_t.append(row[1])

In [ ]:
X[1].shape

In [ ]:
X = np.array(X)
X_train = X.reshape(-1, 256, 256, 3)
y_train = np.array(y)

X_t = np.array(X_t)
X_test = X_t.reshape(-1, 256, 256, 3)
y_test = np.array(y_t)

In [ ]:
label_encoder = LabelEncoder()

# Fit the encoder to your string labels and transform them into integer labels
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
print(y_train_encoded)

class_to_string_mapping = {label: string for label, string in enumerate(label_encoder.classes_)}


In [ ]:
# train_datagen = ImageDataGenerator(
#     shear_range=5,
#     horizontal_flip=True,
#     vertical_flip=True
# )

# batch_size = 32
# # Create a generator from the loaded data
# train_generator = train_datagen.flow(X_train, y_train_encoded, batch_size=batch_size, shuffle=True)


In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(32, (3,3), 1, activation = 'relu', input_shape=(256,256,3)))
model.add(tf.keras.layers.MaxPooling2D(2,2))

model.add(tf.keras.layers.Conv2D(64, (3,3), 1, activation = 'relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2))

model.add(tf.keras.layers.Conv2D(32, (3,3), 1, activation = 'relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(256, activation='relu'))
# model.add(tf.keras.layers.BatchNormalization()) 
model.add(tf.keras.layers.Dense(10,activation='softmax'))
# model.add(tf.keras.layers.Dense(10,activation='sigmoid'))


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()



In [ ]:
batch_size = 32

# train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train_encoded))
# train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size)

# model.fit(train_dataset,batch_size=batch_size, epochs=10, validation_data=(X_test, y_test_encoded))
# test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded, verbose=0)

model.fit(X_train, y_train_encoded,batch_size = batch_size, epochs=25, validation_data=(X_test, y_test_encoded))
test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded, verbose=0)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
plt.figure(figsize=(12, 6))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X_test[i], cmap='gray')
    plt.title(f"Predicted: {tf.argmax(predictions[i]).numpy()}\nActual: {y_test_encoded[i]}")
    plt.axis('off')
plt.show()